In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.set_random_seed(0)

from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [ ]:
mnist = read_data_sets('data', one_hot=True, reshape=False, validation_size=0)

In [ ]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y_true = tf.placeholder(tf.float32, [None, 10])

nH1 = 200
nH2 = 100
nH3 = 60
nH4 = 30

W1 = tf.Variable(tf.truncated_normal([784, nH1] ,stddev=0.1))
B1 = tf.Variable(tf.ones([nH1]) / 10)

W2 = tf.Variable(tf.truncated_normal([nH1, nH2], stddev=0.1))
B2 = tf.Variable(tf.ones([nH2]) / 10)

W3 = tf.Variable(tf.truncated_normal([nH2, nH3], stddev=0.1))
B3 = tf.Variable(tf.ones([nH3]) / 10)

W4 = tf.Variable(tf.truncated_normal([nH3, nH4], stddev=0.1))
B4 = tf.Variable(tf.ones([nH4]) / 10)

W5 = tf.Variable(tf.truncated_normal([nH4, 10], stddev=0.1))
B5 = tf.Variable(tf.ones([10]) / 10)

In [ ]:
pkeep = tf.placeholder(tf.float32)

In [ ]:
# model
X1 = tf.reshape(X, [-1, 784])

Y1 = tf.nn.relu(tf.matmul(X1, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
Y2d = tf.nn.dropout(Y2, pkeep)

Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
Y3d = tf.nn.dropout(Y3, pkeep)

Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
Y4d = tf.nn.dropout(Y4, pkeep)

Y_logits = tf.matmul(Y4d, W5) + B5
Y_pred = tf.nn.softmax(Y_logits)

In [ ]:
# loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Y_logits, 
                                                        labels=Y_true)
cross_entropy = tf.reduce_sum(cross_entropy)

In [ ]:
is_correct = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y_true, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [ ]:
lr = tf.placeholder(tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
from ipywidgets import *
import bqplot.pyplot as plt

n = 2000
acc_fig = plt.figure(title='Accuracy')
x = np.arange(1, n+1)
axes_options = {'x': {'label': 'Iterations', 'num_ticks': 5},
                'y': {'tick_format': '.0%'}}
acc_plot = plt.plot(x, [], display_legend=True, 
                    labels=['Train', 'Test'], colors=['green', 'blue'],
                    axes_options=axes_options)

loss_fig = plt.figure(title='Loss')
loss_plot = plt.plot(np.arange(1, n+1), [], display_legend=True, 
                     labels=['Train', 'Test'], 
                     colors=['green', 'blue'],
                     axes_options={'x': {'label': 'Iterations', 
                                         'num_ticks': 5},
                                   'y': {'tick_format': ','}})
HBox([acc_fig, loss_fig])

In [ ]:
from time import sleep

train_a = []
test_a = []
train_loss = []
test_loss = []
max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 200.0

# 10 epochs
for i in range(1000):
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * np.exp(-i / decay_speed)
    
    # load batch of images and true values
    batch_X, batch_Y = mnist.train.next_batch(100) 
    train_data = {X: batch_X, Y_true: batch_Y, 
                  lr: learning_rate,
                  pkeep: .75}
    
    # train
    sess.run(train_step, feed_dict=train_data)
    a, c = sess.run([accuracy, cross_entropy], feed_dict=train_data)
    train_a.append(a)
    train_loss.append(c * 100)
    
    # run on test data (no dropout on test, pkeep = 1)
    test_data = {X: mnist.test.images, Y_true: mnist.test.labels,
                 pkeep: 1.}
    a, c = sess.run([accuracy, cross_entropy], feed_dict=test_data)
    test_a.append(a)
    test_loss.append(c)
    
    sleep(.001)
    if i % 10 == 0:
        acc_plot.y = [train_a, test_a]
        loss_plot.y = [train_loss, test_loss]